# Introduction

Source of files, analysis schema, 
options:  
- analyzed promoter and enhancer regions
- rare MAF threshold  
- how to treat missing frequencies  
- reference population for binomial test

# Imports

In [92]:
import pandas as pd
from scipy.stats import binom_test
from statsmodels.sandbox.stats.multicomp import multipletests

# Paths to input and output files and 3rd party software

Start by setting paths to input files, output folder and 3rd party software necessary to run this analysis.  
Input files include your vcf file with variants and two interval_list files describing promoters and enhancers active in human brain. Common active promoters and common active enhancers identified and described in Stepniak et al [ref] are provided by default but you can replace them with your own regions files if you wish.  
All intermediate output files will be saved to the output folder defined here.  
If you use the VirtualBox Ubuntu image provided for this analysis the paths to software executables are already set.

In [79]:
### Input files
input_vcf = "data/test_variants_chr16.vcf.gz"
promoter_regions = "data/brain_promoters_active.bed"
enhancer_regions = "data/brain_enhancers_active.bed"

### Output folder
output = "output/"

### Software paths
gatk = "/home/researcher/Programs/gatk-4.1.9.0/gatk"
annovar = "/home/researcher/Programs/annovar/"

In [80]:
#Create output folder
!mkdir $output

mkdir: cannot create directory 'output/': File exists


Bed files with promoter and enhancer regions will be converted to interval_list files required by GATK.

In [81]:
conversion_logs = []
for r, regions in [("promoter", promoter_regions), ("enhancer", enhancer_regions)]:
    command ="%s BedToIntervalList -I %s -O %s -SD %s" % (gatk, regions, regions.replace(".bed", ".interval_list"), input_vcf)
    print(command)
    log = !$command
    conversion_logs.append(log)
    print("Done")

/home/researcher/Programs/gatk-4.1.9.0/gatk BedToIntervalList -I data/brain_promoters_active.bed -O data/brain_promoters_active.interval_list -SD data/test_variants_chr16.vcf.gz
Done
/home/researcher/Programs/gatk-4.1.9.0/gatk BedToIntervalList -I data/brain_enhancers_active.bed -O data/brain_enhancers_active.interval_list -SD data/test_variants_chr16.vcf.gz
Done


Terminal output from 3rd party software is stored in log variables. You can check them if you suspect that something could have gone wrong during the calculations:

In [82]:
conversion_logs[0]

['14:15:15.330 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/researcher/Programs/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so',
 '[Thu Feb 04 14:15:15 CET 2021] BedToIntervalList --INPUT data/brain_promoters_active.bed --SEQUENCE_DICTIONARY data/test_variants_chr16.vcf.gz --OUTPUT data/brain_promoters_active.interval_list --SORT true --UNIQUE false --DROP_MISSING_CONTIGS false --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --GA4GH_CLIENT_SECRETS client_secrets.json --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false',
 'Feb 04, 2021 2:15:15 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine',
 'INFO: Failed to detect whether we are running on Google Compute Engine.',
 '[Thu Feb 04 14:15:15 CET 2021] Executing as researc

# Select biallelic SNPs located in promoters and enhancers

In the first step of the analysis biallelic SNPs located in promoter and enhancer regions are selected from the input .vcf files. Two vcf files are generated in this step: promoter_SNPs.vcf and enhancer_SNPs.vcf

In [83]:
count_before = !$gatk CountVariants -V $input_vcf
print("Number of variants in the input file:", count_before[-4])

Number of variants in the input file: 11088


In [84]:
select_logs = []
count_logs = []
for r, regions in [("promoter", promoter_regions), ("enhancer", enhancer_regions)]:
    command1 ="%s SelectVariants -V %s -L %s --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O %s%s_SNPs.vcf" % (gatk, input_vcf, regions, output, r)
    print(command1)
    log1 = !$command1
    select_logs.append(log1)
    print("Done")
    
    command2 = "%s CountVariants -V %s%s_SNPs.vcf" % (gatk, output, r)
    print(command2)
    log2 = !$command2
    count_logs.append(log2)
    print("Done")
    print("Number of biallelic SNPs in %s regions:" % r, log2[-4])

/home/researcher/Programs/gatk-4.1.9.0/gatk SelectVariants -V data/test_variants_chr16.vcf.gz -L data/brain_promoters_active.bed --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O output/promoter_SNPs.vcf
Done
/home/researcher/Programs/gatk-4.1.9.0/gatk CountVariants -V output/promoter_SNPs.vcf
Done
Number of biallelic SNPs in promoter regions: 3936
/home/researcher/Programs/gatk-4.1.9.0/gatk SelectVariants -V data/test_variants_chr16.vcf.gz -L data/brain_enhancers_active.bed --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O output/enhancer_SNPs.vcf
Done
/home/researcher/Programs/gatk-4.1.9.0/gatk CountVariants -V output/enhancer_SNPs.vcf
Done
Number of biallelic SNPs in enhancer regions: 2446


# Annotate with allele frequencies from gnomAD genome

We will use ANNOVAR [ref] to annotate promoter and enhancer SNPs with population frequencies from the gnomAD genome resource.

In [85]:
annovar_logs = []
for r in ["promoter", "enhancer"]:
    command = "perl %stable_annovar.pl %s%s_SNPs.vcf %shumandb/ -buildver hg38 -remove -protocol gnomad_genome -operation f -nastring . -vcfinput -out %s%s_SNPs" % (annovar, output, r, annovar, output, r)
    print(command)
    log = !$command
    annovar_logs.append(log)
    print("Done")

perl /home/researcher/Programs/annovar/table_annovar.pl output/promoter_SNPs.vcf /home/researcher/Programs/annovar/humandb/ -buildver hg38 -remove -protocol gnomad_genome -operation f -nastring . -vcfinput -out output/promoter_SNPs
Done
perl /home/researcher/Programs/annovar/table_annovar.pl output/enhancer_SNPs.vcf /home/researcher/Programs/annovar/humandb/ -buildver hg38 -remove -protocol gnomad_genome -operation f -nastring . -vcfinput -out output/enhancer_SNPs
Done


In [86]:
!ls -lrth $output/

total 35M
-rw-rw-r-- 1 researcher researcher  471 Feb  4 13:55 promoter_SNPs.hg38_multianno.csv
-rw-rw-r-- 1 researcher researcher  288 Feb  4 13:56 promoter_SNPs.csv
-rw-rw-r-- 1 researcher researcher 4.5M Feb  4 14:15 promoter_SNPs.hg38_multianno.nomissing.vcf
-rw-rw-r-- 1 researcher researcher 2.9M Feb  4 14:15 enhancer_SNPs.hg38_multianno.nomissing.vcf
-rw-rw-r-- 1 researcher researcher 682K Feb  4 14:15 promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf
-rw-rw-r-- 1 researcher researcher 113K Feb  4 14:15 promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf.idx
-rw-rw-r-- 1 researcher researcher 417K Feb  4 14:15 enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf
-rw-rw-r-- 1 researcher researcher 113K Feb  4 14:15 enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf.idx
-rw-rw-r-- 1 researcher researcher 3.6M Feb  4 14:15 promoter_SNPs.vcf
-rw-rw-r-- 1 researcher researcher 117K Feb  4 14:15 promoter_SNPs.vcf.idx
-rw-rw-r-- 1 researcher researc

Annovar has generated two \*.hg38_multianno.vcf files which contain frequency annotations.

## Select SNPs with MAF < 0.01

In the next step we will choose only rare SNPs - those with minor allele frequency (MAF) below 0.01 in all populations included in gnomAD genome.  
  
First we will replace ".", which marks missing MAF values, with "100.". As a result all variants with missing frequency data will be filtered out in the next step. You can motify this behaviour by changing the value which is inserted instead of "." but it must be a float value for the filtering to work properly. For example if you would like to treat missing data as equal to very low frequency you may replace "100." with "0.0".

In [87]:
annotations = ['gnomAD_genome_ALL',
             'gnomAD_genome_AFR',
             'gnomAD_genome_AMR',
             'gnomAD_genome_ASJ',
             'gnomAD_genome_EAS',
             'gnomAD_genome_FIN',
             'gnomAD_genome_NFE',
             'gnomAD_genome_OTH']
for r in ["promoter", "enhancer"]:
    with open('%s%s_SNPs.hg38_multianno.nomissing.vcf' % (output, r), 'w') as o:
        for line in open('%s%s_SNPs.hg38_multianno.vcf' % (output, r)).readlines():    
            for el in annotations:
                if el + '=.' in line:
                    line = line.replace(el + '=.', el + '=100.0')
            o.write(line)

The \*hg38_multianno.nomissing.vcf files contain "." frequency values replaced by 100.0.  

Now we select rare variants and save them in \*.hg38_multianno.nomissing.gnomad_below_0.01.vcf files.

In [88]:
select_rare_logs = []
for r in ["promoter", "enhancer"]:
    command = "%s SelectVariants -V %s%s_SNPs.hg38_multianno.nomissing.vcf" \
            " -select 'gnomAD_genome_ALL < 0.01'" \
            " -select 'gnomAD_genome_AFR < 0.01'" \
            " -select 'gnomAD_genome_AMR < 0.01'" \
            " -select 'gnomAD_genome_ASJ < 0.01'" \
            " -select 'gnomAD_genome_EAS < 0.01'" \
            " -select 'gnomAD_genome_FIN < 0.01'" \
            " -select 'gnomAD_genome_NFE < 0.01'" \
            " -select 'gnomAD_genome_OTH < 0.01'" \
            " -O %s%s_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf" % (gatk, output, r, output, r) 
    log = !$command
    select_rare_logs.append(log)
    print("Done")

Done
Done


Let's check how many variants have been selected.

In [89]:
count_rare_logs = []
for r in ["promoter", "enhancer"]:
    command = "%s CountVariants -V %s%s_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf" % (gatk, output, r)
    print(command)
    log = !$command
    count_rare_logs.append(log)
    print("Done")
    print("Number of rare SNPs in %s regions:" % r, log[-4])

/home/researcher/Programs/gatk-4.1.9.0/gatk CountVariants -V output/promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf
Done
Number of rare SNPs in promoter regions: 474
/home/researcher/Programs/gatk-4.1.9.0/gatk CountVariants -V output/enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf
Done
Number of rare SNPs in enhancer regions: 223


# Choose SNPs enriched in analyzed cohort compared to chosen population

We will now use binomial test to choose SNPs enriched in out analyzed cohort compared to population. Since the variants anayzed in the test example are from Polish population gnomad_NFE (non-Finnish European) population is chosen but you can modify this option according to your needs.  

First we need to reformat vcf files to csv to be able to read them with pandas. For this we will use the VariantsToTable tool from the GATK package. You can specify fields from the vcf which will be present in the csv. Here I choose information about SNP position, REF and ALT alleles, allele counts and frequencies in the analyzed cohort and gnomAD genome frequencies in global population (gnomAD_genome_ALL) and non-Finnish Europeans (gnomAD_genome_NFE).

In [93]:
totable_logs = []
for r in ["promoter", "enhancer"]:
    command = "%s VariantsToTable  -V %s%s_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf " \
    "-F CHROM -F POS -F REF -F ALT -F AC -F AF -F AN " \
    "-F gnomAD_genome_ALL -F gnomAD_genome_NFE " \
    "-O %s%s_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.csv" % (gatk, output, r, output, r)
    print(command)
    log = !$command
    totable_logs.append(log)
    print("Done")

/home/researcher/Programs/gatk-4.1.9.0/gatk VariantsToTable  -V output/promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf -F CHROM -F POS -F REF -F ALT -F AC -F AF -F AN -F gnomAD_genome_ALL -F gnomAD_genome_NFE -O output/promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.csv
Done
/home/researcher/Programs/gatk-4.1.9.0/gatk VariantsToTable  -V output/enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.vcf -F CHROM -F POS -F REF -F ALT -F AC -F AF -F AN -F gnomAD_genome_ALL -F gnomAD_genome_NFE -O output/enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.csv
Done


Read generated csv files with pandas and inspect their contents.

In [94]:
rare_promoter_snps = pd.read_csv("%s/promoter_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.csv" % output, sep = '\t')
rare_promoter_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE,Unnamed: 9
0,chr16,76736,G,A,2,0.043,46,0.000065,0.0001,NaN
1,chr16,138854,A,G,1,0.022,46,0.005400,0.0090,NaN
2,chr16,165546,T,C,1,0.022,46,0.000600,0.0011,NaN
3,chr16,165666,G,A,2,0.043,46,0.001400,0.0022,NaN
4,chr16,228588,A,G,1,0.022,46,0.003900,0.0067,NaN


In [95]:
rare_enhancer_snps = pd.read_csv("%s/enhancer_SNPs.hg38_multianno.nomissing.gnomad_below_0.01.csv" % output, sep = '\t')
rare_enhancer_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE,Unnamed: 9
0,chr16,68497,C,T,3,0.065,46,0.002300,0.0042,NaN
1,chr16,69116,A,G,1,0.022,46,0.000097,0.0002,NaN
2,chr16,338412,T,C,1,0.022,46,0.002600,0.0042,NaN
3,chr16,843059,C,T,1,0.022,46,0.000400,0.0003,NaN
4,chr16,843964,G,T,1,0.022,46,0.003700,0.0048,NaN


Both csv files contain empty "Unnamed:9" column - remove it.

In [104]:
for df in [rare_enhancer_snps, rare_promoter_snps]:
    for col in df.columns:
        if "Unnamed:" in col:
            df.drop(labels = col, axis=1, inplace = True)
rare_enhancer_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE
0,chr16,68497,C,T,3,0.065,46,0.002300,0.0042
1,chr16,69116,A,G,1,0.022,46,0.000097,0.0002
2,chr16,338412,T,C,1,0.022,46,0.002600,0.0042
3,chr16,843059,C,T,1,0.022,46,0.000400,0.0003
4,chr16,843964,G,T,1,0.022,46,0.003700,0.0048


In [105]:
rare_promoter_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE
0,chr16,76736,G,A,2,0.043,46,0.000065,0.0001
1,chr16,138854,A,G,1,0.022,46,0.005400,0.0090
2,chr16,165546,T,C,1,0.022,46,0.000600,0.0011
3,chr16,165666,G,A,2,0.043,46,0.001400,0.0022
4,chr16,228588,A,G,1,0.022,46,0.003900,0.0067


Calculate p-values for one-sided binomial test in which the number of successes is equal to the number of ALT alleles in the cohort (AC), the number of trials is equal to the total number of identified alleles (AN) and probability of success is equal to population frequency of ALT allele (gnomAD_genome_NFE). The alternative hypothesis is that observed frequency is greater than expected.

In [106]:
def calc_binom_pval(row):
        x = row['AC']
        n = row['AN']
        p = float(row['gnomAD_genome_NFE'])        

        return binom_test(x,n,p, alternative = 'greater')  

In [108]:
for df in [rare_enhancer_snps, rare_promoter_snps]:            
    df['binom_pval']=df.apply(calc_binom_pval, axis=1)
    
    #apply correction for multiple hypothesis testing with the Benjamini-Hochberg procedure, use FDR = 0.01
    multipletests_correction = multipletests(df['binom_pval'], alpha=0.01, 
              method='fdr_bh', is_sorted=False, returnsorted=False)
    df['B-H_reject_H0'] = multipletests_correction[0]
    df['corrected_binom_pval'] = multipletests_correction[1]

In [109]:
rare_promoter_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE,binom_pval,B-H_reject_H0,corrected_binom_pval
0,chr16,76736,G,A,2,0.043,46,0.000065,0.0001,0.000010,True,0.000120
1,chr16,138854,A,G,1,0.022,46,0.005400,0.0090,0.340237,False,0.345315
2,chr16,165546,T,C,1,0.022,46,0.000600,0.0011,0.049368,False,0.086077
3,chr16,165666,G,A,2,0.043,46,0.001400,0.0022,0.004697,False,0.017583
4,chr16,228588,A,G,1,0.022,46,0.003900,0.0067,0.265993,False,0.287104


In [110]:
rare_enhancer_snps.head()

,CHROM,POS,REF,ALT,AC,AF,AN,gnomAD_genome_ALL,gnomAD_genome_NFE,binom_pval,B-H_reject_H0,corrected_binom_pval
0,chr16,68497,C,T,3,0.065,46,0.002300,0.0042,0.000983,False,0.011791
1,chr16,69116,A,G,1,0.022,46,0.000097,0.0002,0.009159,False,0.030135
2,chr16,338412,T,C,1,0.022,46,0.002600,0.0042,0.176018,False,0.215016
3,chr16,843059,C,T,1,0.022,46,0.000400,0.0003,0.013707,False,0.037284
4,chr16,843964,G,T,1,0.022,46,0.003700,0.0048,0.198549,False,0.232782


Select SNPs significantly enriched in analyzed cohort at FDR = 0.01.

In [111]:
rare_enriched_promoter_snps = rare_promoter_snps[rare_promoter_snps["B-H_reject_H0"]]
rare_enriched_enhancer_snps = rare_enhancer_snps[rare_enhancer_snps["B-H_reject_H0"]]
print(len(rare_enriched_promoter_snps), "SNPs in promoters are enriched in analyzed cohort.")
print(len(rare_enriched_enhancer_snps), "SNPs in enhancers are enriched in analyzed cohort.")

49 SNPs in promoters are enriched in analyzed cohort.
16 SNPs in enhancers are enriched in analyzed cohort.


# Annotate with predicted TF binding sites

Use motifbreakR package and Hocomoco v11 full database of TF models to identify SNPs which may destroy or create a TF binding site.